# Merge

In [ ]:
import os
import pandas as pd
from pathlib import Path

current_dir = Path('.')

results_folders = [f for f in current_dir.iterdir() if f.is_dir() and f.name.startswith('results_')]

print(f"Found {len(results_folders)} results_ folders:")
for folder in results_folders:
    print(f"  - {folder.name}")
print()

for folder in results_folders:
    csv_files = list(folder.glob('*_evaluation_metrics.csv'))
    
    for csv_file in csv_files:
        model_name = csv_file.stem.replace('_evaluation_metrics', '')
        
        df = pd.read_csv(csv_file)
        
        df['model'] = model_name
        
        globals()[f"{model_name}_df"] = df
        
        print(f"Loaded: {csv_file.name} from {folder.name}")
        print(f"  -> Saved as: {model_name}_df")
        print(f"  -> Shape: {df.shape}")
        print()

model_dfs = [v for k, v in globals().items() if k.endswith('_df') and isinstance(v, pd.DataFrame)]

if model_dfs:
    merged_df = pd.concat(model_dfs, ignore_index=True)
    
    # 결과를 evaluation_metrics.csv로 저장
    output_file = 'evaluation_metrics.csv'
    merged_df.to_csv(output_file, index=False)
    
    print(f"{'='*60}")
    print(f"Merged dataframe saved to: {output_file}")
    print(f"Total shape: {merged_df.shape}")
    print(f"Models included: {sorted(merged_df['model'].unique().tolist())}")
    print(f"{'='*60}")
    
    # 미리보기
    display(merged_df.head(10))
else:
    print("No CSV files found in results_ folders!")

Found 4 results_ folders:
  - results_lightgbm
  - results_lr
  - results_gru
  - results_sarimax

Loaded: lightgbm_evaluation_metrics.csv from results_lightgbm
  -> Saved as: lightgbm_df
  -> Shape: (25, 6)

Loaded: linear_evaluation_metrics.csv from results_lr
  -> Saved as: linear_df
  -> Shape: (25, 6)

Loaded: gru_evaluation_metrics.csv from results_gru
  -> Saved as: gru_df
  -> Shape: (25, 6)

Loaded: sarimax_evaluation_metrics.csv from results_sarimax
  -> Saved as: sarimax_df
  -> Shape: (13, 6)

Merged dataframe saved to: evaluation_metrics.csv
Total shape: (88, 6)
Models included: ['gru', 'lightgbm', 'linear', 'sarimax']


,Feature_set,Embeddings,Dim_reduction,Model,MSE,model
0,A,-,-,LightGBM,21608.745003,lightgbm
1,B,bodyText,orig,LightGBM,20862.665131,lightgbm
2,B,bodyText,pca,LightGBM,20820.148873,lightgbm
3,B,chunking,orig,LightGBM,20862.665131,lightgbm
4,B,chunking,pca,LightGBM,20824.572243,lightgbm
5,B,headlines,orig,LightGBM,20858.985063,lightgbm
6,B,headlines,pca,LightGBM,20764.420328,lightgbm
7,B,paragraphs,orig,LightGBM,20862.665131,lightgbm
8,B,paragraphs,pca,LightGBM,20613.334087,lightgbm
9,C,bodyText,orig,LightGBM,20863.789584,lightgbm
